In [1]:
import fermionic_mbody as fmb
import numpy as np
import openfermion as of
import sparse 

n, m = 4, 2
d, num, pairs = 2*n, 2*m, False
basis_N = fmb.FixedBasis(d=d, num=num, pairs=pairs)
basis = basis_N

rho_1_arrays = fmb.rho_m_gen(basis, 1)
rho_2_arrays = fmb.rho_m_gen(basis, 2)
rho_2_kkbar_arrays = fmb.rho_2_kkbar_gen(basis)
rho_2_block_arrays = fmb.rho_2_block_gen(basis)
rho_2_block_arrays.shape, rho_2_arrays.shape

ρ₂-block: 100%|██████████| 12/12 [00:00<00:00, 68.69it/s]


((16, 16, 70, 70), (28, 28, 70, 70))

In [13]:
[fmb.rho_m_gen(basis, 2) for x in range(5)]


ρ_2: 100%|██████████| 12/12 [00:00<00:00, 29.14it/s]


[<COO: shape=(28, 28, 70, 70), dtype=float64, nnz=6300, fill_value=0.0>,
 <COO: shape=(28, 28, 70, 70), dtype=float64, nnz=6300, fill_value=0.0>,
 <COO: shape=(28, 28, 70, 70), dtype=float64, nnz=6300, fill_value=0.0>,
 <COO: shape=(28, 28, 70, 70), dtype=float64, nnz=6300, fill_value=0.0>,
 <COO: shape=(28, 28, 70, 70), dtype=float64, nnz=6300, fill_value=0.0>]

In [20]:
[rho_m_gen(basis, 2) for x in range(5)]


ρ_2 (V2): 100%|██████████| 28/28 [00:00<00:00, 1214.80it/s]


[<COO: shape=(28, 28, 70, 70), dtype=float64, nnz=6300, fill_value=0.0>,
 <COO: shape=(28, 28, 70, 70), dtype=float64, nnz=6300, fill_value=0.0>,
 <COO: shape=(28, 28, 70, 70), dtype=float64, nnz=6300, fill_value=0.0>,
 <COO: shape=(28, 28, 70, 70), dtype=float64, nnz=6300, fill_value=0.0>,
 <COO: shape=(28, 28, 70, 70), dtype=float64, nnz=6300, fill_value=0.0>]

In [3]:
rho_m_gen(basis, 2).todense() == fmb.rho_m_gen(basis, 2).todense()

ρ_2: 100%|██████████| 12/12 [00:00<00:00, 31.96it/s]


array([[[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  T

In [2]:
# Filename: rho.py
"""
Optimized generators for **m-body reduced density-matrix tensors** ρ(m).
"""

from __future__ import annotations

import itertools
from typing import List, Tuple, Optional, Sequence, Dict

import numpy as np
import openfermion as of
import sparse
from scipy import sparse as sp_sparse 
from multiprocessing import cpu_count
from math import comb
from functools import wraps

# Assuming these modules exist in the same directory based on the prompt context
# We assume the user provides these dependencies.
from fermionic_mbody._parallel import chunked
from fermionic_mbody.basis import FixedBasis
# Imports from _ofsparse are kept ONLY for specialized rho_2_* generators.
from fermionic_mbody._ofsparse import number_preserving_matrix, restrict_sector_matrix, mask_to_index_map



__all__ = [
    "rho_m_gen",
    "rho_2_block_gen",
    "antisymmetrise_block",
    "rho_2_kkbar_gen",
    "rho_m",
]

# ---------------------------------------------------------------------
# helpers (Original helpers kept for compatibility/rho_2_*)
# ---------------------------------------------------------------------
def _ensure_workers(n_workers: int | None) -> int:
    return max(1, n_workers or cpu_count())

def _restrict_last2(tensor, subset):
    # ... (Original implementation) ...
    subset = np.asarray(subset)
    tensor = tensor[(Ellipsis, subset)]
    nd  = tensor.ndim
    idx = [slice(None)] * (nd - 2) + [subset, slice(None)]
    return tensor[tuple(idx)]

def subspace_aware(gen_fn):
    # (Used by rho_2_* functions)
    @wraps(gen_fn)
    def wrapper(basis, *args, **kw):
        if basis.num is None:
             raise ValueError("basis.num must be set.")
        full_dim = comb(basis.d, basis.num)
        if basis.size == full_dim:
            return gen_fn(basis, *args, **kw)

        # This path is slow as it reconstructs the full basis
        from .basis import FixedBasis
        full_N_basis = FixedBasis(basis.d, basis.num, pairs=basis.pairs)
        tensor_full = gen_fn(full_N_basis, *args, **kw)

        # Requires mask_to_index_map from _ofsparse
        mapping = mask_to_index_map(basis.d, basis.num)
        subset  = np.fromiter((mapping[m] for m in basis.num_ele), int)
        return _restrict_last2(tensor_full, subset)
    return wrapper

# =====================================================================
# OPTIMIZED RHO_M_GEN IMPLEMENTATION (V2: Direct Connection Mapping)
# =====================================================================

def calculate_fermionic_sign_sequential(mask_c: int, indices_i: Sequence[int]) -> int:
    """
    Calculates the correct fermionic sign (Jordan-Wigner, Little-Endian convention).
    
    Applies the annihilation string corresponding to indices_i (sorted increasingly) 
    sequentially to the state |mask_c⟩.
    """
    sign = 1
    current_mask = mask_c
    
    # Apply operators sequentially (lowest index first)
    for idx in indices_i:
        # 1. Calculate sign based on parity of electrons before idx
        mask_before = (1 << idx) - 1
        
        # Use int.bit_count() for optimized population count (Python 3.10+).
        # Fallback for older Python versions: bin(current_mask & mask_before).count('1')
        if hasattr(int, 'bit_count'):
            count = (current_mask & mask_before).bit_count()
        else:
            count = bin(current_mask & mask_before).count('1')

        
        if (count & 1) == 1: # Check parity (if odd)
            sign = -sign
            
        # 2. Update the mask (annihilate electron at idx)
        current_mask ^= (1 << idx) # Assumes idx is present (guaranteed by subset iteration)
        
    return sign

def calculate_connections_v2_refined(basis_N: FixedBasis, m_basis: FixedBasis, d: int, basis_N_indices: Dict[int, List[int]]):
    """
    Calculates the connection map using optimized subset iteration.
    Returns map: (r_idx, ii) -> (c_idx, sign).
    """
    N = basis_N.num
    if N is None: return {}, {}, []
    m = m_basis.num
    mask2idx_m = m_basis._mask2idx

    # Pass 1: Identify Nm_masks (using subset iteration)
    # This correctly handles potentially restricted basis_N and m_basis.
    Nm_masks = set()
    for mask_c in basis_N.num_ele:
        indices_c = basis_N_indices[mask_c]
        # Iterate over all combinations of size m (subsets)
        for indices_i in itertools.combinations(indices_c, m):
            # Reconstruct mask_i efficiently
            mask_i = 0
            for idx in indices_i:
                mask_i |= (1 << idx)
            
            # Check if this m-body operator is in the m_basis
            if mask_i in mask2idx_m:
                Nm_masks.add(mask_c ^ mask_i)

    # Create mapping for (N-m) subspace (LE ascending order)
    sorted_Nm_masks = sorted(list(Nm_masks))
    mask2idx_Nm = {mask: idx for idx, mask in enumerate(sorted_Nm_masks)}
    
    # Pass 2: Calculate connections (using subset iteration)
    connections = {}
    for c_idx, mask_c in enumerate(basis_N.num_ele):
        indices_c = basis_N_indices[mask_c]
        for indices_i in itertools.combinations(indices_c, m):
            mask_i = 0
            for idx in indices_i:
                mask_i |= (1 << idx)
            
            if mask_i in mask2idx_m:
                ii = mask2idx_m[mask_i]
                mask_r = mask_c ^ mask_i
                r_idx = mask2idx_Nm[mask_r]
                
                # Calculate sign (Sequential method)
                sign = calculate_fermionic_sign_sequential(mask_c, indices_i)
                
                connections[(r_idx, ii)] = (c_idx, sign)
    
    return connections, mask2idx_Nm, sorted_Nm_masks

def _process_chunk_connections_v2(args):
    """
    Worker that computes RDM elements by iterating over intermediate state connections.
    This avoids all scipy.sparse overhead.
    """
    r_indices_chunk, connections, D_m, coord_dtype = args
    
    # Accumulate results locally
    indices_list = []
    values_list = []

    # Iterate over intermediate states r
    for r_idx in r_indices_chunk:
        # Collect all connections passing through this r_idx
        incoming = {} # ii -> (c_idx, sign_i)
        # This iteration is O(D_m)
        for ii in range(D_m):
            key = (r_idx, ii)
            if key in connections:
                incoming[ii] = connections[key]
        
        # Calculate contributions T_ij(c_j, c_i) = sum_r sign_i * sign_j
        # Complexity: O(K^2) where K is the number of connections for this r_idx.
        
        for ii, (c_idx_i, sign_i) in incoming.items():
            for jj, (c_idx_j, sign_j) in incoming.items():
                
                # Optimization: Exploit Hermiticity T_ijrc = T_jicr*
                # We only calculate the upper triangle (jj >= ii).
                
                if jj >= ii:
                    value = float(sign_i * sign_j)
                    
                    # Store coordinates [ii, jj, c_idx_j, c_idx_i]
                    # Note: c_idx_j is the 'bra' (r index), c_idx_i is the 'ket' (c index).
                    coords = np.array([[ii, jj, c_idx_j, c_idx_i]], dtype=coord_dtype)
                    indices_list.append(coords)
                    values_list.append(np.array([value]))

                    # Add the symmetric part if ii != jj
                    if ii != jj:
                        # [jj, ii, c_idx_i, c_idx_j]
                        coords_sym = np.array([[jj, ii, c_idx_i, c_idx_j]], dtype=coord_dtype)
                        indices_list.append(coords_sym)
                        values_list.append(np.array([value])) # Value is real


    # Concatenate results before returning
    if not indices_list:
        return np.empty((0, 4), dtype=coord_dtype), np.empty(0, dtype=float)
        
    indices_np = np.concatenate(indices_list, axis=0)
    values_np = np.concatenate(values_list, axis=0)
    
    return indices_np, values_np

# ---------------------------------------------------------------------
# generic ρ(m) tensor (Optimized V2)
# ---------------------------------------------------------------------

# No @subspace_aware needed as V2 handles subspaces inherently and efficiently.
def rho_m_gen(
    basis: FixedBasis, m: int, *, n_workers: int | None = None
) -> sparse.COO:
    """
    Build the sparse ρ(m) tensor with indices ``[i, j, r, c]``.
    (Optimized version V2: Direct Connection Mapping with Subset Iteration).
    Adheres to LE convention of FixedBasis.
    """
    if basis.num is None:
         raise ValueError("basis.num must be set for RDM generation.")
         
    # Ensure m_basis respects the same structure/restrictions if applicable
    m_basis = FixedBasis(basis.d, num=m, pairs=basis.pairs) 
    
    D_N = basis.size
    D_m = m_basis.size
    shape = (D_m, D_m, D_N, D_N)

    if m > basis.num or basis.num == 0 or m < 0:
        return sparse.COO([], [], shape=shape)
        
    if m == 0:
        eye = sp_sparse.eye(D_N, dtype=float)
        return sparse.COO(eye).reshape((1, 1, D_N, D_N))

    # Pre-calculate indices for basis_N (Optimization for subset iteration)
    basis_N_indices = {}
    for mask_c in basis_N.num_ele:
        basis_N_indices[mask_c] = [i for i in range(basis.d) if (mask_c >> i) & 1]

    # 1. Calculate connections and define intermediate subspace (Sequential, optimized)
    connections, mask2idx_Nm, sorted_Nm_masks = calculate_connections_v2_refined(
        basis, m_basis, basis.d, basis_N_indices)
    
    D_Nm = len(sorted_Nm_masks)
    if D_Nm == 0:
        return sparse.COO([], [], shape=shape)

    # Memory Optimization: Determine optimal integer type
    max_dim = max(D_m, D_N)
    if max_dim <= np.iinfo(np.int16).max:
        coord_dtype = np.int16 # Sufficient for d=12
    elif max_dim <= np.iinfo(np.int32).max:
        coord_dtype = np.int32
    else:
        coord_dtype = np.int64

    # 2. Parallel computation by iterating over intermediate states
    n_workers = _ensure_workers(n_workers)
    
    # Split the intermediate states (r_indices) into chunks
    r_indices = np.arange(D_Nm)
    # Using more chunks than workers can improve load balancing
    num_chunks = max(n_workers * 4, 1) 
    chunks = np.array_split(r_indices, num_chunks)
    
    # Prepare iterable. The 'connections' dict is shared (read-only) among workers.
    iterable = [(list(chunk), connections, D_m, coord_dtype) 
                for chunk in chunks if len(chunk) > 0]

    results = chunked(
        _process_chunk_connections_v2,
        iterable,
        n_workers=n_workers,
        description=f"ρ_{m} (V2)",
    )

    # 3. Merge results
    idx_chunks = []
    val_chunks = []
    for idx_np, val_np in results:
        if idx_np.size > 0:
            idx_chunks.append(idx_np)
            val_chunks.append(val_np)

    if not idx_chunks:
        return sparse.COO([], [], shape=shape)

    # Final concatenation
    coords_T = np.concatenate(idx_chunks, axis=0).T
    data = np.concatenate(val_chunks, axis=0)
    
    # sparse.COO handles duplicate coordinates by summing them (required by the V2 approach).
    return sparse.COO(coords_T, data, shape=shape)


# ---------------------------------------------------------------------
# specialised ρ₂ blocks and contraction helper
# (These remain unchanged from the original file provided in the prompt)
# ---------------------------------------------------------------------

# The following functions are included exactly as provided in the original prompt's rho.py.
# Note that these still rely on the slower OpenFermion methods and the potentially 
# inconsistent ordering in _ofsparse.py.

def _block_worker(
    args: Tuple[np.ndarray, FixedBasis, np.ndarray]
) -> Tuple[List[List[int]], List[float]]:
    """
    Worker that fills the *pair-scattering* block (Original Implementation)
    """
    chunk, basis, full_set = args
    indices: list[list[int]] = []
    values: list[float] = []

    m_pairs = basis.d // 2  # number of (k, \bar k) pairs

    for idx1 in chunk:
        i, j = divmod(int(idx1), m_pairs)
        # op₂ acts on (j, \bar j) indices
        op2 = of.FermionOperator(((2 * j + 1, 0), (2 * i, 0)))

        for idx2 in full_set:
            k, l = divmod(int(idx2), m_pairs)
            op1 = of.FermionOperator(((2 * k, 1), (2 * l + 1, 1)))

            op = op1 * op2
            mat = number_preserving_matrix(op, basis.d, basis.num)

            rows, cols = mat.nonzero()
            for r, c, v in zip(rows, cols, mat.data):
                indices.append([idx1, idx2, r, c])
                values.append(v)

    return indices, values

@subspace_aware
def rho_2_block_gen(basis: FixedBasis, *, n_workers: int | None = None) -> sparse.COO:
    # ... (Original implementation) ...
    m_pairs = basis.d // 2
    it_set = np.arange(m_pairs**2)
    n_workers = _ensure_workers(n_workers)

    chunks = np.array_split(it_set, n_workers or 0)

    results = chunked(
        _block_worker,
        [(chunk, basis, it_set) for chunk in chunks],
        n_workers=n_workers,
        description="ρ₂-block",
    )

    idx_list, val_list = [], []
    for idx, val in results:
        idx_list.extend(idx)
        val_list.extend(val)

    shape = (m_pairs**2, m_pairs**2, basis.size, basis.size)
    if not idx_list: return sparse.COO([], [], shape=shape)
    coords = np.asarray(idx_list).T
    return sparse.COO(coords, val_list, shape=shape)

def antisymmetrise_block(rho: sparse.COO) -> sparse.COO:
    # ... (Original implementation) ...
    m_pairs = int(round(np.sqrt(rho.shape[0])))
    if m_pairs * m_pairs != rho.shape[0]:
        raise ValueError("ρ has not the expected shape (m², m², …)")

    pairs = [(i, j) for i in range(m_pairs) for j in range(i + 1, m_pairs)]
    n_pairs = len(pairs)

    rows, cols, data = [], [], []
    for r, (i, j) in enumerate(pairs):
        rows.extend([r, r])
        cols.extend([i * m_pairs + j, j * m_pairs + i])
        data.extend([+1 / np.sqrt(2), -1 / np.sqrt(2)])

    P = sparse.COO(np.vstack([rows, cols]), data,
                   shape=(n_pairs, m_pairs ** 2))

    tmp = sparse.tensordot(P, rho, axes=(1, 0))
    rho_bar = sparse.tensordot(tmp, P.T, axes=(1, 0))
    return rho_bar

def _kkbar_worker(
    args: Tuple[np.ndarray, FixedBasis, np.ndarray]
) -> Tuple[List[List[int]], List[float]]:
    # ... (Original implementation) ...
    chunk, basis, full_set = args
    indices: list[list[int]] = []
    values: list[float] = []

    for ii in chunk:
        ii = int(ii)  # pair index for the *row*
        op2 = of.FermionOperator(((2 * ii + 1, 0), (2 * ii, 0)))

        for jj in full_set:
            jj = int(jj)  # pair index for the *column*
            op1 = of.FermionOperator(((2 * jj, 1), (2 * jj + 1, 1)))

            op = op1 * op2
            mat = number_preserving_matrix(op, basis.d, basis.num)

            rows, cols = mat.nonzero()
            for r, c, v in zip(rows, cols, mat.data):
                indices.append([ii, jj, r, c])
                values.append(v)

    return indices, values

@subspace_aware
def rho_2_kkbar_gen(basis: FixedBasis, *, n_workers: int | None = None) -> sparse.COO:
    # ... (Original implementation) ...
    m_pairs = basis.d // 2
    it_set = np.arange(m_pairs)
    n_workers = _ensure_workers(n_workers)

    chunks = np.array_split(it_set, n_workers or 0)

    results = chunked(
        _kkbar_worker,
        [(chunk, basis, it_set) for chunk in chunks],
        n_workers=n_workers,
        description="ρ₂-k k̄",
    )

    idx_list, val_list = [], []
    for idx, val in results:
        idx_list.extend(idx)
        val_list.extend(val)

    shape = (m_pairs, m_pairs, basis.size, basis.size)
    if not idx_list: return sparse.COO([], [], shape=shape)
    coords = np.asarray(idx_list).T
    return sparse.COO(coords, val_list, shape=shape)

def rho_m(state: np.ndarray, rho_arrays: sparse.COO) -> sparse.COO:
    # ... (Original implementation) ...
    ndim = state.ndim
    if ndim == 1:  # single ket
        # Ensure complex conjugation for <Psi| if state is complex
        return sparse.einsum("k,ijkl,l->ij", state.conj(), rho_arrays, state)

    if ndim == 2:  # density matrix
        # Handle the m=0 case specifically if needed, otherwise einsum works
        if rho_arrays.shape[0] == 1 and rho_arrays.shape[1] == 1 and m == 0:
             # RDM(0) is just the trace of the state
             trace = np.trace(state)
             return sparse.COO(trace).reshape((1, 1))
        return sparse.einsum("ijkl,kl->ij", rho_arrays, state)

    if ndim == 3:  # batch of kets
        return sparse.einsum("bk,ijkl,bl->bij", state.conj(), rho_arrays, state)

    raise ValueError("`state` must be a ket, density matrix, or batch thereof")

In [4]:
def pair_condensate_state(basis: fmb.FixedBasis, m_pairs: int) -> np.ndarray:
    d = basis.d
    assert d % 2 == 0
    n_pairs = d // 2
    assert 0 < m_pairs <= n_pairs

    vec = np.zeros(basis.size, complex)

    # iterate over every subset of m time-reversed pairs
    from itertools import combinations

    for combo in combinations(range(n_pairs), m_pairs):
        bitmask = 0
        for k in combo:
            bitmask |= 1 << (2 * k)      # c†_k
            bitmask |= 1 << (2 * k + 1)  # c†_{k̄}
        idx = int(np.where(basis.num_ele == bitmask)[0])
        vec[idx] = 1.0

    vec /= np.linalg.norm(vec)
    return vec.real

def a_dag_state(basis: fmb.FixedBasis, sk_arr: np.ndarray):
    # Generamos los operadores A^\dag
    op = of.FermionOperator.zero()
    for k in range(0, basis.d//2):
        op += sk_arr[k] * of.FermionOperator((2*k, 1)) * of.FermionOperator((2*k+1, 1))
    op = np.prod([op for _ in range(basis.m//2)])
    vect = basis.opr_to_vect(op).real
    print(op)
    return vect * 1/np.linalg.norm(vect)

def partial_traspose(rho, perm):
    d = basis.d//2
    rho_p = np.zeros((d**2, d**2))
    for m in range(d**2):
        for n in range(d**2):
            # Escrimos los índices i j k l en términos de m n
            i, j = m//d, m % d
            k, l = n//d, n % d
            indices = (i, j, k, l)
            ip, jp, kp, lp = tuple(indices[p] for p in perm)
            rho_p[m,n] = rho[ip*d + jp, kp*d + lp]
    return rho_p



def slater_state(basis: fmb.FixedBasis, occupied) -> np.ndarray:
    bitmask = sum(1 << i for i in occupied)
    idx = int(np.where(basis.num_ele == bitmask)[0])
    vec = np.zeros(basis.size, complex)
    vec[idx] = 1.0
    return vec

In [5]:
# Uniforme
sk, lam = 1/np.sqrt(n) * np.ones(n), -1*m*(n+1-2*m)/(n*(n-1))
# Random (solo m=1)
#seed = np.random.uniform(0, 1, n)
#sk, lam = seed/np.linalg.norm(seed), 0

vect = a_dag_state(basis, sk)
#vect = slater_state(basis, np.random.randint(0, basis.d, basis.m))

rho = fmb.rho_m(vect, rho_2_block_arrays)

#rho_pt = partial_traspose(rho, [0, 3, 2, 1]) 
#rho_as = fmb.rho.antisymmetrise_block(rho_pt)
#au = lambda x: np.sort(np.linalg.eigvals(x).real)

#print(sk)
#au(rho_as), lam

0.25 [0^ 1^ 0^ 1^] +
0.25 [0^ 1^ 2^ 3^] +
0.25 [0^ 1^ 4^ 5^] +
0.25 [0^ 1^ 6^ 7^] +
0.25 [2^ 3^ 0^ 1^] +
0.25 [2^ 3^ 2^ 3^] +
0.25 [2^ 3^ 4^ 5^] +
0.25 [2^ 3^ 6^ 7^] +
0.25 [4^ 5^ 0^ 1^] +
0.25 [4^ 5^ 2^ 3^] +
0.25 [4^ 5^ 4^ 5^] +
0.25 [4^ 5^ 6^ 7^] +
0.25 [6^ 7^ 0^ 1^] +
0.25 [6^ 7^ 2^ 3^] +
0.25 [6^ 7^ 4^ 5^] +
0.25 [6^ 7^ 6^ 7^]
